# Gamma Index Analysis Tutorial

This tutorial is presenting the gamma index (GI) analysis methods implemented in FREDtools. The GI computation is performed by an external C++ code provided as a shared library and developed by Angelo Schiavi from Sapienza University of Rome. This tutorial is based on the example files Ref.mhd and Eval.mhd available in the examples of the [GitHub repository](https://github.com/jasqs/FREDtools), which describe reference and evaluation dose distributions, respectively.

In [1]:
# Import of FREDtools and other useful modules
import numpy as np
import matplotlib.pyplot as plt

import fredtools as ft
print("Current FREDtools version", ft.__version__)


Current FREDtools version 0.6.50


# Reading example file

In [2]:
imgRef = ft.readMHD("Ref.mhd", displayInfo=True) # voxel size [2.5 2.5 2.4]
imgEval = ft.readMHD("Eval.mhd", displayInfo=True) # voxel size [1.5 1.5 1.5]

### readMHD ###
# 3D image describing volume (3D)
# dims (xyz) =  [ 93 116 121]
# voxel size [mm] =  [2.5 2.5 2.4]
# origin [mm]     =  [-114.5117188 -356.4882813 -718.3      ]
# x-spatial voxel centre [mm] =  [  -114.511719,  -112.011719, ...,   112.988281,   115.488281 ]
# y-spatial voxel centre [mm] =  [  -356.488281,  -353.988281, ...,   -71.488281,   -68.988281 ]
# z-spatial voxel centre [mm] =  [  -718.300000,  -715.900000, ...,  -432.700000,  -430.300000 ]
# x-spatial extent [mm] =  [  -115.761719 ,   116.738281 ] =>   232.500000
# y-spatial extent [mm] =  [  -357.738281 ,   -67.738281 ] =>   290.000000
# z-spatial extent [mm] =  [  -719.500000 ,  -429.100000 ] =>   290.400000
# volume = 19580220.00 mm3  =>  19.58 l
# voxel volume = 15.00 mm3  =>  15.00 ul
# data type:  64-bit float 
# range: from  0.0  to  39.123884725879094 
# sum = 1181653.291965024 , mean = 0.9052400524343118 ( 4.738983902347923 ) 
# non-zero (dose=0)  voxels  = 115142 (8.82%) => 1.73 l 
# non-air (HU>-1000)

# calculate gamma index map (basic)

In [3]:
# Many options can be changed when calculating the GI. Refer to the documentation for more details

"""
By default it is only needed to specify the reference and evaluation images and the calculation criteria, 
like dose-difference, distance-to-agreement and dose-cutoff.
"""
imgGI=ft.calcGammaIndex(imgRef, imgEval, DD=2, DTA=2, DCO=0.05, displayInfo=True)


"""
To calculate the GI statistics use the following function which returns a dictionary with the statistical parameters.
"""
statGI=ft.getGIstat(imgGI, displayInfo=True)

DTA,DD,DDT,DCO : 2 2 L 5

Num of active voxel to be tested: 61631
Interpolate dose values using neighboring voxels

maxDose ref : 39.1239 Gy
maxDose eval: 39.4316 Gy

global normalisation dose : 39.1239 Gy

DCO: 1.95619 Gy
Using local DD for agreement: 2 %

Evaluation grid step size: 0.2 mm  -> num of fractional steps per DTA = 10

Maximum gamma searched for: inf
Nref Neval 1305348 5684160
Num of skimmed voxels: 0

Executing the kernel on 12 threads ...
	searching from 0 to DTA
[  10  20  30  40  50  60  70  80  90 100]
 ........................................
	searching from 1*DTA to 2*DTA
[  10  20  30  40  50  60  70  80  90 100]
 ........................................
	searching from 2*DTA to 3*DTA
[  10  20  30  40  50  60  70  80  90 100]
 ........................................
	searching from 3*DTA to 4*DTA
[  10  20  30  40  50  60  70  80  90 100]
 ........................................
	searching from 4*DTA to 5*DTA
[  10  20  30  40  50  60  70  80  90 100]
 .........

In [4]:
# The GI parameters are saved as tags to the img and will be saved to HMD header
ft.displayImageInfo(imgGI)

### displayImageInfo ###
# 3D image describing volume (3D)
# dims (xyz) =  [ 93 116 121]
# voxel size [mm] =  [2.5 2.5 2.4]
# origin [mm]     =  [-114.5117188 -356.4882813 -718.3      ]
# x-spatial voxel centre [mm] =  [  -114.511719,  -112.011719, ...,   112.988281,   115.488281 ]
# y-spatial voxel centre [mm] =  [  -356.488281,  -353.988281, ...,   -71.488281,   -68.988281 ]
# z-spatial voxel centre [mm] =  [  -718.300000,  -715.900000, ...,  -432.700000,  -430.300000 ]
# x-spatial extent [mm] =  [  -115.761719 ,   116.738281 ] =>   232.500000
# y-spatial extent [mm] =  [  -357.738281 ,   -67.738281 ] =>   290.000000
# z-spatial extent [mm] =  [  -719.500000 ,  -429.100000 ] =>   290.400000
# volume = 19580220.00 mm3  =>  19.58 l
# voxel volume = 15.00 mm3  =>  15.00 ul
# data type:  32-bit float 
# range: from  -1.0  to  10.804873 
# sum = -1210478.2 , mean = -0.92732227 ( 0.34434244 ) 
# non-zero (dose=0)  voxels  = 1305348 (100.00%) => 19.58 l 
# non-air (HU>-1000) voxels  = 13053

# calculate gamma index map (advanced)

In [5]:
"""
It is possible to define many other parameters. Refer to the documentation for more details. A few examples below.
"""
print("The same configuration as in the basic example but run on 10 threads instead of all available.")
imgGI=ft.calcGammaIndex(imgRef, imgEval, DD=2, DTA=2, DCO=0.05, DDType="local",
                        stepSize=10, fractionalStepSize=True, 
                        mode="gamma", CPUNo=10, displayInfo=False)
statGI=ft.getGIstat(imgGI, displayInfo=True)

print("Dose difference set to global and only the pass rate calculated.")
imgGI=ft.calcGammaIndex(imgRef, imgEval, DD=2, DTA=2, DCO=0.05, DDType="global",
                        stepSize=10, fractionalStepSize=True, 
                        mode="pass-rate", CPUNo="auto", displayInfo=False)
statGI=ft.getGIstat(imgGI, displayInfo=True)

print("The same as the previous but with the fractional step size increased.")
imgGI=ft.calcGammaIndex(imgRef, imgEval, DD=2, DTA=2, DCO=0.05, DDType="global",
                        stepSize=20, fractionalStepSize=True, 
                        mode="pass-rate", CPUNo="auto", displayInfo=False)
statGI=ft.getGIstat(imgGI, displayInfo=True)

print("Gamma calculation mode with local normalisaiton set to 30.")
imgGI=ft.calcGammaIndex(imgRef, imgEval, DD=2, DTA=2, DCO=0.05, DDType="local", globalNorm=30, 
                        stepSize=10, fractionalStepSize=True, 
                        mode="gamma", CPUNo="auto", displayInfo=False)
statGI=ft.getGIstat(imgGI, displayInfo=True)


The same configuration as in the basic example but run on 10 threads instead of all available.
### getGIstat ###
# GIPR: 85.42
# mean/std: 0.54 / 0.50
# min/max: 0.00 / 10.80
#################
Dose difference set to global and only the pass rate calculated.
### getGIstat ###
# GIPR: 87.85
#################
The same as the previous but with the fractional step size increased.
### getGIstat ###
# GIPR: 88.10
#################
Gamma calculation mode with local normalisaiton set to 30.
### getGIstat ###
# GIPR: 85.53
# mean/std: 0.54 / 0.51
# min/max: 0.00 / 10.80
#################
